In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import time
import math
import pickle

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
rf_model = pickle.load(open('./models/phil-mickelson-stroke-rf.pkl', 'rb'))
split_clf_model = pickle.load(open('./models/phil-mickelson-stroke-rf-clf.pkl', 'rb'))
split_reg_model = pickle.load(open('./models/phil-mickelson-stroke-rf-reg.pkl', 'rb'))

In [3]:
locations = {
    '0': 'Tee Box',
    '1': 'Fairway',
    '2': 'Fringe',
    '3': 'Green',
    '4': 'Intermediate Rough',
    '5': 'Primary Rough',
    '6': 'Green Side Bunker',
    '7': 'Fairway/Waste Bunker',
    '8': 'Tree/Rock/Dirt',
    '9': 'Cart Path',
    '10': 'Native Area',
    '11': 'Water',
    '12': 'Unknown',
    '-1': 'IN THE HOLE'
}

def predict_shot_result_rf(d, hl, sl):
    X_sim = pd.DataFrame(data=d, index=[0])
    y_sim_pred = rf_model.predict(X_sim)
    
    location = int(round(y_sim_pred[0][0], 0))
    distance_to_pin_yards = round(y_sim_pred[0][1] / 36.0, 0)
    distance_to_pin_inches = y_sim_pred[0][1]
    
    print('Predicted End Location', locations[str(location)], location)
    print('Predicted Distance Hit (yds)', distance_to_pin_yards)
    print('Predicted Distance Hit (in)', distance_to_pin_inches)
    print('Yardage Remaining', round((hl-sl-distance_to_pin_inches) / 36.0, 0))
    print()
    
    return location, distance_to_pin_inches


def predict_shot_result_split(d, hl, sl):
    X_sim = pd.DataFrame(data=d, index=[0])
    y_sim_pred_clf = split_clf_model.predict(X_sim)
    y_sim_pred_reg = split_reg_model.predict(X_sim)
    
    location = int(round(y_sim_pred_clf[0], 0))
    distance_to_pin_yards = round(y_sim_pred_reg[0] / 36.0, 0)
    distance_to_pin_inches = y_sim_pred_reg[0]
    
    print('Predicted End Location', locations[str(location)], location)
    print('Predicted Distance Hit (yds)', distance_to_pin_yards)
    print('Predicted Distance Hit (in)', distance_to_pin_inches)
    print('Yardage Remaining', round((hl-sl-distance_to_pin_inches) / 36.0, 0))
    print()
    
    return location, distance_to_pin_inches

In [10]:
print('Prediction without SG values using 1 model')
print('-'*100)

loc = 1
dist = 503 * 36
hole_length = 503*36
shot_lengths = 0

for i in range(5):
    d = {
        'Shot': i+1,
        'From Location(Scorer)': loc,
        'Distance to Pin': hole_length - shot_lengths,
        'Lie': 1, # Good
        'Elevation': 1, # With
        'Slope': 1, # Level
        'Total SG Putting': 0, 
        'Avg SG Putting': 0,
        'Total SG OTT': 0, 
        'Avg SG OTT': 0, 
        'Total SG Approach': 0, 
        'Avg SG Approach': 0, 
        'Total SG Around the Green': 0, 
        'Avg SG Around the Green': 0,
        'Fwy Firmness': 2, # Firm
        'Fwy Height': 0.36000001430511475,
        'Grn Firmness': 2, # Firm
        'Grn Height': 0.10000000149011612,
        'Rough Height': 2.25,
        'Stimp': 12.50,
        'Fwy Width 250': 28.00,
        'Fwy Width 275': 29.00,
        'Fwy Width 300': 28.00,
        'Fwy Width 325': 0.00,
        'Fwy Width 350': 0.00,
        'Actual 250 Distance': 25.00,
        'Actual 275 Distance': 28.00,
        'Actual 300 Distance': 30.00,
        'Actual 325 Distance': 0.00,
        'Actual 350 Distance': 0.00,
        'Par': 5,
        'Yardage': 503
    }
    loc, dist = predict_shot_result_rf(d, hole_length, shot_lengths)
    shot_lengths = shot_lengths + dist
    if loc == -1:
        break

Prediction without SG values using 1 model
----------------------------------------------------------------------------------------------------
Predicted End Location Green 3
Predicted Distance Hit (yds) 269.0
Predicted Distance Hit (in) 9685.84
Yardage Remaining 234.0

Predicted End Location Primary Rough 5
Predicted Distance Hit (yds) 228.0
Predicted Distance Hit (in) 8207.25
Yardage Remaining 6.0

Predicted End Location Green 3
Predicted Distance Hit (yds) 7.0
Predicted Distance Hit (in) 235.73
Yardage Remaining -1.0

Predicted End Location IN THE HOLE -1
Predicted Distance Hit (yds) 0.0
Predicted Distance Hit (in) 1.0
Yardage Remaining -1.0



In [9]:
print('Prediction with current SG values using 2 models')
print('-'*100)

loc = 1
dist = 503 * 36
hole_length = 503*36
shot_lengths = 0

for i in range(5):
    d = {
        'Shot': i+1,
        'From Location(Scorer)': loc,
        'Distance to Pin': hole_length - shot_lengths,
        'Lie': 1, # Good
        'Elevation': 1, # With
        'Slope': 1, # Level
        'Total SG Putting': 14.441, 
        'Avg SG Putting': 0.656,
        'Total SG OTT': 4.830, 
        'Avg SG OTT': 0.220, 
        'Total SG Approach': 6.695, 
        'Avg SG Approach': 0.304, 
        'Total SG Around the Green': 3.367, 
        'Avg SG Around the Green': 0.153,
        'Fwy Firmness': 2, # Firm
        'Fwy Height': 0.36000001430511475,
        'Grn Firmness': 2, # Firm
        'Grn Height': 0.10000000149011612,
        'Rough Height': 2.25,
        'Stimp': 12.50,
        'Fwy Width 250': 28.00,
        'Fwy Width 275': 29.00,
        'Fwy Width 300': 28.00,
        'Fwy Width 325': 0.00,
        'Fwy Width 350': 0.00,
        'Actual 250 Distance': 25.00,
        'Actual 275 Distance': 28.00,
        'Actual 300 Distance': 30.00,
        'Actual 325 Distance': 0.00,
        'Actual 350 Distance': 0.00,
        'Par': 5,
        'Yardage': 503
    }
    loc, dist = predict_shot_result_split(d, hole_length, shot_lengths)
    shot_lengths = shot_lengths + dist
    if loc == -1:
        break

Prediction with current SG values using 2 models
----------------------------------------------------------------------------------------------------
Predicted End Location Fairway 1
Predicted Distance Hit (yds) 277.0
Predicted Distance Hit (in) 9989.734666666667
Yardage Remaining 226.0

Predicted End Location Green 3
Predicted Distance Hit (yds) 222.0
Predicted Distance Hit (in) 7977.56
Yardage Remaining 4.0

Predicted End Location Green 3
Predicted Distance Hit (yds) 4.0
Predicted Distance Hit (in) 158.97
Yardage Remaining -1.0

Predicted End Location IN THE HOLE -1
Predicted Distance Hit (yds) 0.0
Predicted Distance Hit (in) 1.0
Yardage Remaining -1.0



In [8]:
print('Prediction without SG values using 2 models')
print('-'*100)

loc = 1
dist = 503 * 36
hole_length = 503*36
shot_lengths = 0

for i in range(5):
    d = {
        'Shot': i+1,
        'From Location(Scorer)': loc,
        'Distance to Pin': hole_length - shot_lengths,
        'Lie': 1, # Good
        'Elevation': 1, # With
        'Slope': 1, # Level
        'Total SG Putting': 0, 
        'Avg SG Putting': 0,
        'Total SG OTT': 0, 
        'Avg SG OTT': 0, 
        'Total SG Approach': 0, 
        'Avg SG Approach': 0, 
        'Total SG Around the Green': 0, 
        'Avg SG Around the Green': 0,
        'Fwy Firmness': 2, # Firm
        'Fwy Height': 0.36000001430511475,
        'Grn Firmness': 2, # Firm
        'Grn Height': 0.10000000149011612,
        'Rough Height': 2.25,
        'Stimp': 12.50,
        'Fwy Width 250': 28.00,
        'Fwy Width 275': 29.00,
        'Fwy Width 300': 28.00,
        'Fwy Width 325': 0.00,
        'Fwy Width 350': 0.00,
        'Actual 250 Distance': 25.00,
        'Actual 275 Distance': 28.00,
        'Actual 300 Distance': 30.00,
        'Actual 325 Distance': 0.00,
        'Actual 350 Distance': 0.00,
        'Par': 5,
        'Yardage': 503
    }
    loc, dist = predict_shot_result_split(d, hole_length, shot_lengths)
    shot_lengths = shot_lengths + dist
    if loc == -1:
        break

Prediction without SG values using 2 models
----------------------------------------------------------------------------------------------------
Predicted End Location Fairway 1
Predicted Distance Hit (yds) 272.0
Predicted Distance Hit (in) 9807.254666666668
Yardage Remaining 231.0

Predicted End Location Primary Rough 5
Predicted Distance Hit (yds) 224.0
Predicted Distance Hit (in) 8061.23
Yardage Remaining 7.0

Predicted End Location Green 3
Predicted Distance Hit (yds) 7.0
Predicted Distance Hit (in) 250.58
Yardage Remaining -0.0

Predicted End Location IN THE HOLE -1
Predicted Distance Hit (yds) 0.0
Predicted Distance Hit (in) 1.0
Yardage Remaining -0.0

